# **Word2Vec練習，以維基百科為例**
自然語言處理NLP(Natural Language Processing) 是人工智慧最主要的應用之一，近年來在深度學習領域取得重大進展，其中Word2Vec模型扮演極重要的角色。

Word2Vec是 2013 年由 Tomas Mikolov 等人所提出，透過大量資料的訓練，將深度學習訓練的權重向量，用來表達字詞的語意，讓語意相似的單字可以有較近的距離。
本範例以2020/3/1所下載的中文維基百科為範例，用來建立Word2Vec語意模型並且預測相關離的字詞

（由於Wiki的語料庫含有簡體中文，將會轉換為繁體後再進行模型訓練）

In [2]:
# 安裝簡轉繁工具 - opencc-python-reimplemented
!pip install opencc-python-reimplemented
# 使用wget指令，從wiki獲取語料庫
!wget https://dumps.wikimedia.org/zhwiki/20200301/zhwiki-20200301-pages-articles-multistream1.xml-p1p162886.bz2

# import gensim 對維基百科處理的類 WikiCorpus
from gensim.corpora import WikiCorpus

wiki_corpus = WikiCorpus('zhwiki-20200301-pages-articles-multistream1.xml-p1p162886.bz2', dictionary={})
next(iter(wiki_corpus.get_texts()))[:10]

# 初始化text_num（詞的數量）
text_num = 0

# 將維基裡的每篇文章轉換位1行text文字，並且去掉標點符號等內容
with open('wiki_text.txt', 'w', encoding='utf-8') as f:
    for text in wiki_corpus.get_texts():
        f.write(' '.join(text)+'\n')
        text_num += 1
        if text_num % 10000 == 0:
            print('{} articles processed.'.format(text_num))

    print('{} articles processed.'.format(text_num))

# import jieba 用以之後分詞
import jieba
from opencc import OpenCC


# Initial
# 初始化
# 將wiki的語料庫簡轉繁且分詞後，再將其儲存，檔名為seg.txt
cc = OpenCC('s2t')
train_data = open('wiki_text.txt', 'r', encoding='utf-8').read()
train_data = cc.convert(train_data)
train_data = jieba.lcut(train_data)
train_data = [word for word in train_data if word != '']
train_data = ' '.join(train_data)
open('seg.txt', 'w', encoding='utf-8').write(train_data)

from gensim.models import word2vec


# Settings
# 設定各項參數
seed = 666           # 亂數種子，用於random時，與詞向量有關
sg = 0               # 設定訓練算法，為0時採用CBOW算法、為1時採用skip-gram算法
window_size = 10     # 周圍詞彙要看多少範圍
vector_size = 100    # 轉成向量的維度
min_count = 1        # 詞頻少於 min_count 之詞彙不會參與訓練
workers = 8          # 訓練的並行數量
epochs = 5           # 訓練的迭代次數
batch_words = 10000  # 每次給予多少詞彙量訓練

# 讀入上方已經分詞好的語料庫，並且也設定好各項參數
train_data = word2vec.LineSentence('seg.txt')
model = word2vec.Word2Vec(
    train_data,
    min_count=min_count,
    size=vector_size,
    workers=workers,
    iter=epochs,
    window=window_size,
    sg=sg,
    seed=seed,
    batch_words=batch_words
)

model.save('word2vec.model')


from gensim.models import word2vec

string = '微生物'
model = word2vec.Word2Vec.load('word2vec.model')
print(string)

for item in model.wv.most_similar(string):
    print(item)

     |████████████████████████████████| 491kB 40.3MB/s 
  Created wheel for opencc-python-reimplemented: filename=opencc_python_reimplemented-0.1.5-py2.py3-none-any.whl size=485664 sha256=8d451874e7669142031dd283ede4b8ad51e2229bcbdc157ed89d63e75d0f136d
  Stored in directory: /root/.cache/pip/wheels/36/a0/10/888b9ac7f10154caaa6a73270b1f085e0a7b241baa672bbe49
Successfully built opencc-python-reimplemented
--2020-04-24 13:31:54--  https://dumps.wikimedia.org/zhwiki/20200301/zhwiki-20200301-pages-articles-multistream1.xml-p1p162886.bz2
Resolving dumps.wikimedia.org (dumps.wikimedia.org)... 208.80.154.7, 2620:0:861:1:208:80:154:7
Connecting to dumps.wikimedia.org (dumps.wikimedia.org)|208.80.154.7|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 172586252 (165M) [application/octet-stream]
Saving to: ‘zhwiki-20200301-pages-articles-multistream1.xml-p1p162886.bz2’

zhwiki-20200301-pag 100%[===================>] 164.59M  4.80MB/s    in 34s     

2020-04-24 13:32:28 (4.

Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
Loading model cost 0.725 seconds.
Prefix dict has been built successfully.
/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


微生物
('細菌', 0.8789703845977783)
('菌', 0.8646241426467896)
('代謝', 0.8630419969558716)
('藻類', 0.8562043905258179)
('真菌', 0.8535659313201904)
('胰島素', 0.8511801362037659)
('免疫', 0.8462857604026794)
('致病', 0.8450480699539185)
('抗藥性', 0.8393200039863586)
('激素', 0.8370355367660522)


/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):
